# ArangoDB NetworkX Adapter Getting Started Guide  

<a href="https://colab.research.google.com/github/arangoml/networkx-adapter/blob/master/examples/ArangoDB_NetworkX_Adapter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![arangodb](https://github.com/arangoml/networkx-adapter/blob/master/examples/assets/logos/ArangoDB_logo.png?raw=1)
![networkX](https://github.com/arangoml/networkx-adapter/blob/master/examples/assets/logos/networkx_logo.svg?raw=1) 

Version: 3.0.0

Objective: Export Graphs from [ArangoDB](https://www.arangodb.com/), a multi-model Graph Database, to [NetworkX](https://networkx.github.io/), the swiss army knife for graph analysis in python, and vice-versa.

# Setup

In [ ]:
%%capture
!git clone -b oasis_connector --single-branch https://github.com/arangodb/interactive_tutorials.git
!git clone -b 3.0.0 --single-branch https://github.com/arangoml/networkx-adapter.git
!rsync -av networkx-adapter/examples/ ./ --exclude=.git
!rsync -av interactive_tutorials/ ./ --exclude=.git
!pip3 install adbnx_adapter==3.0.0
!pip3 install matplotlib
!pip3 install pyArango

In [ ]:
import json
import oasis
import networkx as nx
import matplotlib.pyplot as plt


from adbnx_adapter.adapter import ADBNX_Adapter
from adbnx_adapter.controller import ADBNX_Controller
from adbnx_adapter.typings import Json, ArangoMetagraph, NxId, NxData

# Create a Temporary ArangoDB Instance

In [ ]:
# Request temporary instance from the managed ArangoDB Cloud Oasis.
con = oasis.getTempCredentials()

# Connect to the db via the python-arango driver
python_arango_db_driver = oasis.connect_python_arango(con)

# (Alternative) Connect to the db via the pyArango driver
# pyarango_db_driver = oasis.connect(con)[con['dbName']]

print()
print("https://{}:{}".format(con["hostname"], con["port"]))
print("Username: " + con["username"])
print("Password: " + con["password"])
print("Database: " + con["dbName"])

Feel free to use to above URL to checkout the UI!

# Data Import

We will use an Fraud Detection example graph, explained in more detail in this [interactive notebook](https://colab.research.google.com/github/joerg84/Graph_Powered_ML_Workshop/blob/master/Fraud_Detection.ipynb).

*Note the included arangorestore will only work on Linux system, if you want to run this notebook on a different OS please consider using the appropriate arangorestore from the [Download area](https://www.arangodb.com/download-major/).*

In [ ]:
%%capture
!chmod -R 755 ./tools
!./tools/arangorestore -c none --server.endpoint http+ssl://{con["hostname"]}:{con["port"]} --server.username {con["username"]} --server.database {con["dbName"]} --server.password {con["password"]} --default-replication-factor 3  --input-directory "data/fraud_dump"
!./tools/arangorestore -c none --server.endpoint http+ssl://{con["hostname"]}:{con["port"]} --server.username {con["username"]} --server.database {con["dbName"]} --server.password {con["password"]} --default-replication-factor 3  --input-directory "data/imdb_dump"

# Create Graph

The graph we will be using in the following looks as follows:

![networkX](https://github.com/arangoml/networkx-adapter/blob/master/examples/assets/fraud_graph.jpeg?raw=1) 

In [ ]:
edge_definitions = [
    {
        "edge_collection": "accountHolder",
        "from_vertex_collections": ["customer"],
        "to_vertex_collections": ["account"],
    },
    {
        "edge_collection": "transaction",
        "from_vertex_collections": ["account"],
        "to_vertex_collections": ["account"],
    },
]

name = "fraud-detection"
python_arango_db_driver.delete_graph(name, ignore_missing=True)
fraud_graph = python_arango_db_driver.create_graph(name, edge_definitions=edge_definitions)

print("Graph Setup done.")
print(fraud_graph)

Feel free to visit the ArangoDB UI using the above link and login data and check the Graph!

# Create Adapter

Connect the ArangoDB_Networkx_Adapter to our temp ArangoDB cluster:

In [ ]:
adbnx_adapter = ADBNX_Adapter(con)

# ArangoDB to NetworkX



## Via ArangoDB Graph

In [ ]:
# Define graph name
graph_name = "fraud-detection"

# Create NetworkX graph from ArangoDB graph
nx_g = adbnx_adapter.arangodb_graph_to_networkx(graph_name)

# You can also provide valid Python-Arango AQL query options to the command above, like such:
# nx_g = adbnx_adapter.arangodb_graph_to_networkx(graph_name, ttl=1000, stream=True)
# See more here: https://docs.python-arango.com/en/main/specs.html#arango.aql.AQL.execute

# Show graph data
print(nx_g.nodes(data=True))
print(nx_g.edges(data=True))
nx.draw(nx_g, with_labels=True)

## Via ArangoDB Collections

In [ ]:
# Define collection
vertex_collections = {"account", "bank", "branch", "Class", "customer"}
edge_collections = {"accountHolder", "Relationship", "transaction"}

# Create NetworkX graph from ArangoDB collections
nx_g = adbnx_adapter.arangodb_collections_to_networkx("fraud-detection", vertex_collections, edge_collections)

# You can also provide valid Python-Arango AQL query options to the command above, like such:
# nx_g = adbnx_adapter.arangodb_collections_to_networkx("fraud-detection", vertex_collections, edge_collections, ttl=1000, stream=True)
# See more here: https://docs.python-arango.com/en/main/specs.html#arango.aql.AQL.execute

# Show graph data
print(nx_g.nodes(data=True))
print(nx_g.edges(data=True))
nx.draw(nx_g, with_labels=True)

## Via ArangoDB Metagraph

In [ ]:
# Define metagraph
fraud_detection_metagraph = {
    "vertexCollections": {
        "account": {"Balance", "account_type", "customer_id", "rank"},
        "bank": {"Country", "Id", "bank_id", "bank_name"},
        "branch": {"City", "Country", "Id", "bank_id", "branch_id", "branch_name"},
        "Class": {"concrete", "label", "name"},
        "customer": {"Name", "Sex", "Ssn", "rank"},
    },
    "edgeCollections": {
        "accountHolder": {},
        "Relationship": {"label", "name", "relationshipType"},
        "transaction": {"transaction_amt", "sender_bank_id", "receiver_bank_id"},
    },
}

# Create NetworkX Graph from attributes
nx_g = adbnx_adapter.arangodb_to_networkx('fraud-detection',  fraud_detection_metagraph)

# You can also provide valid Python-Arango AQL query options to the command above, like such:
# nx_g = adbnx_adapter.arangodb_to_networkx('fraud-detection',  fraud_detection_metagraph, ttl=1000, stream=True)
# See more here: https://docs.python-arango.com/en/main/specs.html#arango.aql.AQL.execute

# Show graph data
print(nx_g.nodes(data=True))
print(nx_g.edges(data=True))

nx.draw(nx_g, with_labels=True)

## Via ArangoDB Metagraph with a custom controller

In [ ]:
# Define metagraph
imdb_metagraph = {
    "vertexCollections": {"Users": {"Age", "Gender"}, "Movies": {}},
    "edgeCollections": {"Ratings": {"Rating"}},
}

class IMDB_ADBNX_Controller(ADBNX_Controller):
    """ArangoDB-NetworkX controller.

    Responsible for controlling how nodes & edges are handled when
    transitioning from ArangoDB to NetworkX, and vice-versa.

    You can derive your own custom ADBNX_Controller, but it is not
    necessary for Homogeneous graphs.
    """
    # We re-define how vertex pre-insertion should be treated, specifically for the IMDB dataset.
    def _prepare_arangodb_vertex(self, adb_vertex: Json, col: str) -> NxId:
        """Prepare an ArangoDB vertex before it gets inserted into the NetworkX
        graph.

        Given an ArangoDB vertex, you can modify it before it gets inserted
        into the NetworkX graph, and/or derive a custom node id for networkx to use.
        In most cases, it is only required to return the ArangoDB _id of the vertex.

        :param vertex: The ArangoDB vertex object to (optionally) modify.
        :type vertex: dict
        :param col: The ArangoDB collection the vertex belongs to.
        :type col: str
        :return: The ArangoDB _id attribute of the vertex.
        :rtype: str
        """
        adb_vertex["bipartite"] = 0 if col == "Users" else 1 # New bipartite attribute logic
        return super()._prepare_arangodb_vertex(adb_vertex, col) # Return ArangoDB _id

    # We're not interested in re-defining pre-insertion handling for edges, so we leave it alone
    # def _prepare_arangodb_edge(self, adb_edge: Json, col: str) -> NxId:
    #   return super()._prepare_arangodb_edge(edge, collection)

# Instantiate the adapter
imdb_adbnx_adapter = ADBNX_Adapter(con, IMDB_ADBNX_Controller())

# Create NetworkX Graph from metagraph using the custom IMDB_ArangoDB_Networx_Adapter
nx_g = imdb_adbnx_adapter.arangodb_to_networkx("IMDBGraph", imdb_metagraph)

# You can also provide valid Python-Arango AQL query options to the command above, like such:
# nx_g = imdb_adbnx_adapter.arangodb_to_networkx("IMDBGraph", imdb_metagraph, ttl=1000, stream=True)
# See more here: https://docs.python-arango.com/en/main/specs.html#arango.aql.AQL.execute

# Show graph data
print(nx_g.nodes(data=True))
# print(nx_g.edges(data=True)) # (will exceed IOPub data rate)
# nx.draw(nx_g, with_labels=True) # (will exceed IOPub data rate)

# NetworkX to ArangoDB

## Example 1: NetworkX Grid Graph

In [ ]:
# Load the nx graph & draw
grid_nx_g = nx.grid_2d_graph(5, 5)
nx.draw(grid_nx_g, with_labels=True)

# We must provide edge definitions to create the ArangoDB graph
# Since this graph is Homogeneous, we only need one edge definition.
edge_definitions = [
    {
        "edge_collection": "to",
        "from_vertex_collections": ["Grid_Node"],
        "to_vertex_collections": ["Grid_Node"],
    }
]

# Create the ArangoDB graph
name = "Grid_1"
python_arango_db_driver.delete_graph(name, drop_collections=True, ignore_missing=True)
grid_adb_g = adbnx_adapter.networkx_to_arangodb(name, grid_nx_g, edge_definitions, batch_size=1)

print('\n--------------------')
print("https://{}:{}".format(con["hostname"], con["port"]))
print("Username: " + con["username"])
print("Password: " + con["password"])
print("Database: " + con["dbName"])
print('--------------------\n')
print(f"Inspect the graph here: https://tutorials.arangodb.cloud:8529/_db/{con['dbName']}/_admin/aardvark/index.html#graph/{name}")
print(f"View the original graph here: https://networkx.org/documentation/stable/auto_examples/basic/plot_read_write.html#sphx-glr-auto-examples-basic-plot-read-write-py)")

## Example 2: NetworkX Football Graph

In [ ]:
import io
import zipfile
import urllib.request as urllib

def get_football_graph():
    url = "http://www-personal.umich.edu/~mejn/netdata/football.zip"
    sock = urllib.urlopen(url)
    s = io.BytesIO(sock.read())
    sock.close()
    zf = zipfile.ZipFile(s)
    gml = zf.read("football.gml").decode()
    gml_list = gml.split("\n")[1:]
    return nx.parse_gml(gml_list)

football_nx_g = get_football_graph()
nx.draw(football_nx_g, with_labels=True)

# We must provide edge definitions to create the ArangoDB graph
# Since this graph is Homogeneous, we only need one edge definition.
edge_definitions = [
    {
        "edge_collection": "played",
        "from_vertex_collections": ["Football_Team"],
        "to_vertex_collections": ["Football_Team"],
    }
]

class Football_ADBNX_Controller(ADBNX_Controller):
    """ArangoDB-NetworkX controller.

    Responsible for controlling how nodes & edges are handled when
    transitioning from ArangoDB to NetworkX, and vice-versa.

    You can derive your own custom ADBNX_Controller, but it is not
    necessary for Homogeneous graphs.
    """
    def _keyify_networkx_node(self, nx_node_id: NxId, nx_node: NxData, col: str) -> str:
        """Given a NetworkX node, derive its valid ArangoDB key.

        NOTE: You must override this function if you want to create custom ArangoDB _key
        values for your NetworkX nodes or if your NetworkX graph does NOT comply to
        ArangoDB standards (i.e the node IDs are not formatted
        like "{collection}/{key}"). For more  info, see the **keyify_nodes**
        parameter of ADBNX_Adapter.networkx_to_arangodb()

        :param nx_node: The NetworkX node object.
        :type nx_node: dict
        :param col: The ArangoDB collection the node belongs to.
        :type col: str
        :return: A valid ArangoDB _key value.
        :rtype: str
        """
        # Since our NetworkX Football nodes have an id of type string, we can use the existing helper function.
        adb_v_key: str = self._string_to_arangodb_key_helper(str(nx_node_id))
        return adb_v_key

# Instantiate the adapter
football_adbnx_adapter = ADBNX_Adapter(con, Football_ADBNX_Controller())

# Create the ArangoDB graph
name = "Football"
python_arango_db_driver.delete_graph(name, drop_collections=True, ignore_missing=True)
football_adb_g = football_adbnx_adapter.networkx_to_arangodb(name, football_nx_g, edge_definitions, keyify_nodes=True)

print('\n--------------------')
print("https://{}:{}".format(con["hostname"], con["port"]))
print("Username: " + con["username"])
print("Password: " + con["password"])
print("Database: " + con["dbName"])
print('--------------------\n')
print(f"Inspect the graph here: https://tutorials.arangodb.cloud:8529/_db/{con['dbName']}/_admin/aardvark/index.html#graph/{name}")
print(f"View the original graph here: https://networkx.org/documentation/stable/auto_examples/graph/plot_football.html#sphx-glr-auto-examples-graph-plot-football-py)")

# Full Cycles

## Example 1: ArangoDB ➡ NetworkX ➡ ArangoDB (with existing collections)

In [ ]:
name = "fraud-detection"

# Start from ArangoDB graph
original_fraud_adb_g = python_arango_db_driver.graph(name)
edge_definitions = original_fraud_adb_g.edge_definitions()

# Create NetworkX graph from ArangoDB graph
fraud_nx_g = adbnx_adapter.arangodb_graph_to_networkx(name)
nx.draw(fraud_nx_g, with_labels=True)

# Modify the NetworkX graph
for _, node in fraud_nx_g.nodes(data=True):
    node["new_vertex_data"] = ["new", "vertex", "data", "here"]

for _, _, edge in fraud_nx_g.edges(data=True):
    edge["new_edge_data"] = ["new", "edge", "data", "here"]

# Re-use existing graph's edge definitions to overwrite existing graph
# Keify nodes & edges to keep the same key values as original (this is optional)
updated_fraud_adb_g = adbnx_adapter.networkx_to_arangodb(
    name,
    fraud_nx_g,
    edge_definitions,
    keyify_nodes=True,
    keyify_edges=True,
)

print(f"Inspect the overwritten graph here: https://tutorials.arangodb.cloud:8529/_db/{con['dbName']}/_admin/aardvark/index.html#graph/{name}")

## Example 2: ArangoDB ➡ NetworkX ➡ ArangoDB (with new collections)

In [ ]:
name = "fraud-detection"

# Start from ArangoDB graph
original_fraud_adb_g = python_arango_db_driver.graph(name) 

# Create NetworkX graph from ArangoDB graph
fraud_nx_g = adbnx_adapter.arangodb_graph_to_networkx(name)
nx.draw(fraud_nx_g, with_labels=True)

# We must provide edge definitions to create the ArangoDB graph
# Since this graph is Heterogeneous, we must provide multiple edge definitions.
edge_definitions = [
    {
        "edge_collection": "accountHolder_new",
        "from_vertex_collections": ["customer_new"],
        "to_vertex_collections": ["account_new"],
    },
    {
        "edge_collection": "transaction_new",
        "from_vertex_collections": ["account_new"],
        "to_vertex_collections": ["account_new"],
    },
]

class Fraud_ADBNX_Controller(ADBNX_Controller):
  """ArangoDB-NetworkX controller.

  Responsible for controlling how nodes & edges are handled when
  transitioning from ArangoDB to NetworkX, and vice-versa.

  You can derive your own custom ADBNX_Controller, but it is not
  necessary for Homogeneous graphs.
  """
  # Since we are dealing with a Heterogeneous, we must implement _identify_networkx_node().
  def _identify_networkx_node(self, nx_node_id: NxId, nx_node: NxData) -> str:
    """Given a NetworkX node, identify what ArangoDB collection it should belong to.

    NOTE: You must override this function if your NetworkX graph is NOT Homogeneous
    or does NOT comply to ArangoDB standards (i.e the node IDs are not formatted
    like "{collection}/{key}").

    :param nx_node_id: The NetworkX ID of the node.
    :type nx_node_id: adbnx_adapter.typings.NxId
    :param nx_node: The NetworkX node object.
    :type nx_node: adbnx_adapter.typings.NxData
    :return: The ArangoDB collection name
    :rtype: str
    """
    adb_vertex_id: str = str(nx_node_id)
    return adb_vertex_id.split("/")[0] + "_new"

  # Since we are dealing with a Heterogeneous, we must implement _identify_networkx_edge().
  def _identify_networkx_edge(self, nx_edge: NxData, from_nx_node: NxData, to_nx_node: NxData) -> str:
    """Given a NetworkX edge, and its pair of nodes, identify what ArangoDB
    collection should it belong to.

    NOTE: You must override this function if your NetworkX graph is NOT Homogeneous
    or does NOT comply to ArangoDB standards
    (i.e the edge IDs are not formatted like "{collection}/{key}").

    :param nx_edge: The NetworkX edge object.
    :type nx_edge: adbnx_adapter.typings.NxData
    :param from_nx_node: The NetworkX node object representing the edge source.
    :type from_nx_node: adbnx_adapter.typings.NxData
    :param to_nx_node: The NetworkX node object representing the edge destination.
    :type to_nx_node: adbnx_adapter.typings.NxData
    :return: The ArangoDB collection name
    :rtype: str
    """
    adb_vertex_id: str = str(nx_edge["_id"])
    return adb_vertex_id.split("/")[0] + "_new"

fraud_adbnx_adapter = ADBNX_Adapter(con, Fraud_ADBNX_Controller())

# Create a new ArangoDB graph from NetworkX graph
new_name = name + "_new"
python_arango_db_driver.delete_graph(new_name, drop_collections=True, ignore_missing=True)
# Keify nodes & edges to keep the same key values as original (this is optional)
new_fraud_adb_g = fraud_adbnx_adapter.networkx_to_arangodb(
    new_name,
    fraud_nx_g,
    edge_definitions,
    keyify_nodes=True,
    keyify_edges=True,
)

print(f"Inspect the new graph here: https://tutorials.arangodb.cloud:8529/_db/{con['dbName']}/_admin/aardvark/index.html#graph/{new_name}")
print(f"View the original graph here: https://tutorials.arangodb.cloud:8529/_db/{con['dbName']}/_admin/aardvark/index.html#graph/{name}")


## Example 3: NetworkX ➡ ArangoDB ➡ NetworkX

In [ ]:
# Load the nx graph
original_grid_nx_g = nx.grid_2d_graph(5, 5)
print(original_grid_nx_g.nodes(data=True))
print(original_grid_nx_g.edges(data=True))

# We must provide edge definitions to create the ArangoDB graph
# Since this graph is Homogeneous, we only need one edge definition.
edge_definitions = [
    {
        "edge_collection": "to_v2",
        "from_vertex_collections": ["Grid_Node_v2"],
        "to_vertex_collections": ["Grid_Node_v2"],
    }
]

# Re-introduce the Grid controller class
class Grid_ADBNX_Controller(ADBNX_Controller):
    """ArangoDB-NetworkX controller.

    Responsible for controlling how nodes & edges are handled when
    transitioning from ArangoDB to NetworkX, and vice-versa.

    You can derive your own custom ADBNX_Controller, but it is not
    necessary for Homogeneous graphs.
    """
    def _prepare_arangodb_vertex(self, adb_vertex: Json, col: str) -> NxId:
        """Prepare an ArangoDB vertex before it gets inserted into the NetworkX
        graph.

        Given an ArangoDB vertex, you can modify it before it gets inserted
        into the NetworkX graph, and/or derive a custom node id for networkx to use.
        In most cases, it is only required to return the ArangoDB _id of the vertex.

        :param adb_vertex: The ArangoDB vertex object to (optionally) modify.
        :type adb_vertex: adbnx_adapter.typings.Json
        :param col: The ArangoDB collection the vertex belongs to.
        :type col: str
        :return: The ArangoDB _id attribute of the vertex.
        :rtype: str
        """
        nx_node_id = tuple(
            int(n)
            for n in tuple(
                adb_vertex["_key"],
            )
        )
        return nx_node_id

    def _keyify_networkx_node(self, nx_node_id: NxId, nx_node: NxData, col: str) -> str:
        """Given a NetworkX node, derive its valid ArangoDB key.

        NOTE: You must override this function if you want to create custom ArangoDB _key
        values for your NetworkX nodes or if your NetworkX graph does NOT comply to
        ArangoDB standards (i.e the node IDs are not formatted
        like "{collection}/{key}"). For more  info, see the **keyify_nodes**
        parameter of ADBNX_Adapter.networkx_to_arangodb()

        :param nx_node_id: The NetworkX node id.
        :type nx_node_id: adbnx_adapter.typings.NxId
        :param nx_node: The NetworkX node object.
        :type nx_node: adbnx_adapter.typings.NxData
        :param col: The ArangoDB collection the node belongs to.
        :type col: str
        :return: A valid ArangoDB _key value.
        :rtype: str
        """
        adb_v_key: str = self._tuple_to_arangodb_key_helper(nx_node_id)  # type: ignore
        return adb_v_key

# Re-instantiate the Grid adapter class
grid_adbnx_adapter = ADBNX_Adapter(con, Grid_ADBNX_Controller())

# Delete the Grid graph if it already exists in ArangoDB
name = "Grid_2"
python_arango_db_driver.delete_graph(name, drop_collections=True, ignore_missing=True)

# Create the ArangoDB graph
grid_adbnx_adapter.networkx_to_arangodb(name, original_grid_nx_g, edge_definitions, keyify_nodes=True)

# Create the NetworkX graph from the ArangoDB graph
new_grid_nx_g = grid_adbnx_adapter.arangodb_graph_to_networkx(name)

# Draw the new graph
nx.draw(new_grid_nx_g, with_labels=True)
print(new_grid_nx_g.nodes(data=True))
print(new_grid_nx_g.edges(data=True))